## Konfiguracja ollama

In [1]:
import ollama

ollama_client = ollama.Client(host='http://host.docker.internal:11434')

# all-minilm:l6-v2
# all-minilm:l12-v2

MODEL = "all-minilm:l6-v2"

embedding_size = len(ollama_client.embed(MODEL,'lorem ipsum').embeddings[0])
print(f"Długość osadzeń: {embedding_size}")


Długość osadzeń: 384


## Stworzenie bazy danych SQLite z rozszerzeniami sqlite-vec i FTS5

In [2]:
import sqlite3
import sqlite_vec
from contextlib import contextmanager

DB_FILENAME = "search.db"

@contextmanager
def sqlite_connection():
    conn = sqlite3.connect(DB_FILENAME)
    try:
        conn.enable_load_extension(True)
        sqlite_vec.load(conn)
        # db.load_extension('fts5')
        conn.enable_load_extension(False)

        conn.execute('PRAGMA journal_mode=WAL')

        yield conn
        conn.commit()
    except Exception:
        conn.rollback()
        raise
    finally:
        conn.execute('PRAGMA wal_checkpoint(FULL)')
        conn.execute('PRAGMA journal_mode=DELETE')

        conn.close()

In [3]:
import os

for db_file in [DB_FILENAME,DB_FILENAME + "-shm",DB_FILENAME+"-wal",DB_FILENAME+"-journal"]:
    if os.path.exists(db_file):
        os.remove(db_file)

with sqlite_connection() as conn:
    vec_version, = conn.execute("select vec_version()").fetchone()
    print(f"vec_version={vec_version}")

    conn.execute("""
        CREATE TABLE articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            section TEXT,
            content TEXT,
            link TEXT UNIQUE,
            published_date INTEGER
        );
    """)

    conn.execute("""
        CREATE VIRTUAL TABLE article_embeddings USING vec0 (
            embedding FLOAT[{embedding_size}],
            article_id INTEGER NOT NULL,
            field_name TEXT NOT NULL
        );
    """.format(embedding_size=embedding_size))

    conn.execute("""
        CREATE VIRTUAL TABLE articles_fts USING fts5(
            title,
            content,
            content='articles',
            content_rowid='id'
        )
    """)

vec_version=v0.1.6


## Załadowanie danych wejściowych do bazy danych

### Załadowanie artykułów

In [4]:
import datasets

with sqlite_connection() as conn:
    cursor = conn.cursor()

    # dataset_names = ["RealTimeData/bbc_news_april_2023", "RealTimeData/bbc_news_may_2023", "RealTimeData/bbc_news_june_2023", "RealTimeData/bbc_news_july_2023"]
    dataset_names = ["RealTimeData/bbc_news_week1_july_2023"]
    # dataset_names = ["RealTimeData/bbc_news_may_2023"]

    for dataset_name in dataset_names:
        print("Loading dataset " + dataset_name + "...")
        
        dataset = datasets.load_dataset(dataset_name, split="train")
        for row in dataset:
            cursor.execute("""
                INSERT INTO articles (title, section, content, link, published_date)
                VALUES (?, ?, ?, ?, ?)
                ON CONFLICT(link) DO NOTHING
            """, [row['title'], row['section'], row['content'], row['link'], row['published_date']])
        
        print("Dataset loaded!")

Loading dataset RealTimeData/bbc_news_week1_july_2023...
Dataset loaded!


In [5]:
with sqlite_connection() as conn:
    row_count = conn.execute("SELECT count() AS c FROM articles").fetchone()[0]
    print('Loaded {row_count} rows of data!'.format(row_count=row_count))

Loaded 400 rows of data!


### Wygenerowanie i załadowanie osadzeń

In [6]:
with sqlite_connection() as conn:
    read_cursor = conn.cursor()
    write_cursor = conn.cursor()

    i = 0
    for row in read_cursor.execute("SELECT id, title, content FROM articles"):
        embeddings = ollama_client.embed(model=MODEL, input=[row[1], row[2]]).embeddings
        
        write_cursor.execute("""
            INSERT INTO article_embeddings (embedding, article_id, field_name) VALUES (?, ?, "title"), (?, ?, "content")
        """, [sqlite_vec.serialize_float32(embeddings[0]), row[0], sqlite_vec.serialize_float32(embeddings[1]), row[0]])
    
        i+=1
        if i%50 == 0:
            print("Processed {n} records".format(n=i))

Processed 50 records
Processed 100 records
Processed 150 records
Processed 200 records
Processed 250 records
Processed 300 records
Processed 350 records
Processed 400 records


### Wygenerowanie indeksów do wyszukiwania tekstowego

In [7]:
with sqlite_connection() as conn:
    conn.cursor().execute("INSERT INTO articles_fts(articles_fts) VALUES ('rebuild')")

# Wyszukiwanie

In [8]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

QUERY = "football"

## Wyszukiwanie przy użyciu rozszerzenia sqlite-vec

In [9]:
query_embedding = ollama_client.embed(model=MODEL, input=QUERY).embeddings[0]
query_embedding

[-0.011797418,
 0.045702837,
 -0.0386044,
 -0.0200696,
 0.027311921,
 0.04728293,
 0.10229847,
 0.036840454,
 0.07121883,
 0.034751423,
 -0.06534077,
 -0.05168748,
 -0.013812505,
 0.045669865,
 -0.0101220645,
 -0.030236294,
 -0.054922275,
 -0.12538438,
 -0.02856294,
 -0.106140785,
 -0.061026823,
 0.05844485,
 -0.018008547,
 0.003943575,
 -0.010908782,
 0.026352674,
 -0.011113954,
 0.03268371,
 -0.083455905,
 -0.11986337,
 -0.007516774,
 0.017205179,
 0.027626343,
 0.022373127,
 -0.116218224,
 0.030152362,
 0.025277182,
 -0.019324848,
 0.040997192,
 0.051350407,
 -0.06183952,
 -0.11994863,
 0.007807213,
 0.022286529,
 0.045900017,
 0.0858527,
 0.07445762,
 0.009661759,
 0.015457882,
 0.034951266,
 0.052366946,
 0.04636365,
 -0.08101915,
 0.036451425,
 0.09797119,
 0.0045311144,
 -0.0075007863,
 0.05616813,
 -0.02587305,
 0.013388171,
 0.04837162,
 0.019516699,
 -0.04153011,
 0.017527593,
 0.008526648,
 -0.025089664,
 -0.03451827,
 0.027112078,
 -0.012576807,
 -0.00013855804,
 0.02293668

### Podobieństwo cosinusowe - osobne pola

In [10]:
cosine_search_df = None

with sqlite_connection() as conn:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT 
            a.id,
            a.title,
            a.section,
            a.link,
            vec_distance_cosine(ae.embedding, ?) AS distance,
            ae.field_name
        FROM articles a
        JOIN article_embeddings ae ON a.id = ae.article_id
        ORDER BY distance;
    """, [sqlite_vec.serialize_float32(query_embedding)])

    cosine_search_df = pd.DataFrame(cursor, columns=["Id", "Tytuł", "Sekcja", "Link", "Dystans", "Pole"])

cosine_search_df

,Id,Tytuł,Sekcja,Link,Dystans,Pole
0,44,England 0-0 Portugal: Lionesses draw with Portugal in World Cup send-off - BBC Sport,None,http://www.bbc.co.uk/sport/football/66065939,0.624792,title
1,510,England U21 1-0 Spain U21: England win Under-21 Euros for first time in 39 years - BBC Sport,None,http://www.bbc.co.uk/sport/football/66127349,0.644827,title
2,200,Hannah Dingley: Forest Green Rovers name first female boss of a men's professional football side - BBC Sport,None,http://www.bbc.co.uk/sport/football/66105649,0.683806,title
3,44,England 0-0 Portugal: Lionesses draw with Portugal in World Cup send-off - BBC Sport,None,http://www.bbc.co.uk/sport/football/66065939,0.724464,content
4,510,England U21 1-0 Spain U21: England win Under-21 Euros for first time in 39 years - BBC Sport,None,http://www.bbc.co.uk/sport/football/66127349,0.735059,content
...,...,...,...,...,...,...
795,261,"Bank account closures probe must be fast tracked, says minister - BBC News",Business,http://www.bbc.co.uk/news/business-66111090,1.073010,title
796,246,Nxivm cult: US actor Allison Mack released early from prison - BBC News,US & Canada,http://www.bbc.co.uk/news/world-us-canada-66111026,1.073779,content
797,512,"Dublin hotel dig unearths 1,000-year-old burial site - BBC News",Northern Ireland,http://www.bbc.co.uk/news/uk-northern-ireland-66137475,1.079824,content
798,11,Golden Ears Park: Missing Canadian teen found after 54-hour search - BBC News,US & Canada,http://www.bbc.co.uk/news/world-us-canada-66076558,1.083125,content


### Podobieństwo cosinusowe - ważony wynik z wielu pól

In [11]:
cosine_weight_search_df = None

with sqlite_connection() as conn:
    TITLE_WEIGHT = 0.7

    cursor = conn.cursor()

    cursor.execute("""
        WITH field_similarities AS (
            SELECT 
                article_id AS id,
                field_name,
                (2 - vec_distance_cosine(embedding, :embedding)) / 2 AS similarity
            FROM article_embeddings
        ),
        article_similarities AS (
            SELECT 
                id,
                MAX(CASE WHEN field_name = 'title' THEN similarity END) * :title_weight AS title_weighted_similarity,
                (
                    MAX(CASE WHEN field_name = 'title' THEN similarity END) * :title_weight +
                    MAX(CASE WHEN field_name = 'content' THEN similarity END) * (1 - :title_weight)
                ) AS weighted_similarity
            FROM field_similarities
            GROUP BY id
        )
        SELECT
            a.id AS id,
            a.title AS title,
            a.section AS section,
            a.link AS link,
            asi.weighted_similarity AS similarity,
            PRINTF("%.1f%%", (asi.title_weighted_similarity / asi.weighted_similarity) * 100) AS title_contribution_percent,
            PRINTF("%.1f%%", ((asi.weighted_similarity - asi.title_weighted_similarity) / asi.weighted_similarity) * 100) AS content_contribution_percent
        FROM articles a
        LEFT JOIN article_similarities asi ON a.id = asi.id
        ORDER BY similarity DESC;
    """, {
        'embedding': sqlite_vec.serialize_float32(query_embedding),
        'title_weight': TITLE_WEIGHT
    })

    cosine_weight_search_df = pd.DataFrame(cursor, columns=["Id", "Tytuł", "Sekcja", "Link", "Ważony wynik (0-1)", "Udział tytułu", "Udział zawartości"])

cosine_weight_search_df

,Id,Tytuł,Sekcja,Link,Ważony wynik (0-1),Udział tytułu,Udział zawartości
0,44,England 0-0 Portugal: Lionesses draw with Portugal in World Cup send-off - BBC Sport,None,http://www.bbc.co.uk/sport/football/66065939,0.672653,71.6%,28.4%
1,510,England U21 1-0 Spain U21: England win Under-21 Euros for first time in 39 years - BBC Sport,None,http://www.bbc.co.uk/sport/football/66127349,0.664052,71.4%,28.6%
2,200,Hannah Dingley: Forest Green Rovers name first female boss of a men's professional football side - BBC Sport,None,http://www.bbc.co.uk/sport/football/66105649,0.645022,71.4%,28.6%
3,257,"Wimbledon 2023: Just Stop Oil protesters interrupt play twice, jigsaws taken off sale - BBC Sport",None,http://www.bbc.co.uk/sport/tennis/66041547,0.620694,70.4%,29.6%
4,136,Ukraine war: The lethal minefields holding up Kyiv's counter-offensive - BBC News,Europe,http://www.bbc.co.uk/news/world-europe-66080652,0.615590,71.8%,28.2%
...,...,...,...,...,...,...,...
395,338,Giant stone artefacts found on rare Ice Age site in Kent - BBC News,Kent,http://www.bbc.co.uk/news/uk-england-kent-66112136,0.482039,70.3%,29.7%
396,261,"Bank account closures probe must be fast tracked, says minister - BBC News",Business,http://www.bbc.co.uk/news/business-66111090,0.480905,67.5%,32.5%
397,512,"Dublin hotel dig unearths 1,000-year-old burial site - BBC News",Northern Ireland,http://www.bbc.co.uk/news/uk-northern-ireland-66137475,0.480572,71.3%,28.7%
398,11,Golden Ears Park: Missing Canadian teen found after 54-hour search - BBC News,US & Canada,http://www.bbc.co.uk/news/world-us-canada-66076558,0.479755,71.3%,28.7%


## Wyszukiwanie hybrydowe - scalanie wyników metodą Reciprocal Rank Fusion (RRF)

In [12]:
hybrid_search_df = None

with sqlite_connection() as conn:
    cursor = conn.cursor()

    cursor.execute("""
        WITH vector_similarities AS (
            SELECT 
                article_id AS id,
                field_name,
                (2 - vec_distance_cosine(embedding, :embedding)) / 2 AS similarity
            FROM article_embeddings
        ),
        vector_ranks AS (
            SELECT 
                id,
                ROW_NUMBER() OVER (ORDER BY (
                    MAX(CASE WHEN field_name = 'title' THEN similarity END) * :vector_title_weight +
                    MAX(CASE WHEN field_name = 'content' THEN similarity END) * (1 - :vector_title_weight)
                ) DESC) AS rank
            FROM vector_similarities
            GROUP BY id
        ),
        text_scores_title AS (
            SELECT 
                rowid AS id,
                rank * :text_title_multiplier AS score
            FROM articles_fts 
            WHERE articles_fts
            MATCH ('title:' || :query)
        ),
        text_scores_content AS (
            SELECT 
                rowid AS id,
                rank AS score
            FROM articles_fts 
            WHERE articles_fts
            MATCH ('content:' || :query)
        ),
        text_ranks AS (
            SELECT
                a.id AS id,
                ROW_NUMBER() OVER (ORDER BY COALESCE(tst.score, 0) + COALESCE(tsc.score, 0) DESC) AS rank
            FROM articles a
            LEFT JOIN text_scores_title tst ON a.id = tst.id
            LEFT JOIN text_scores_content tsc ON a.id = tsc.id
            WHERE COALESCE(tst.id, tsc.id) IS NOT NULL
        )
        SELECT
            a.id AS id,
            a.title AS title,
            a.section AS section,
            a.link AS link,
            (COALESCE(1.0 / (:rrf_k + vr.rank), 0) * :vector_weight) + (COALESCE(1.0 / (:rrf_k + tr.rank), 0) * (1 - :vector_weight)) AS score,
            PRINTF("%.1f%%", (COALESCE(1.0 / (:rrf_k + vr.rank), 0) * :vector_weight) / ((COALESCE(1.0 / (:rrf_k + vr.rank), 0) * :vector_weight) + (COALESCE(1.0 / (:rrf_k + tr.rank), 0) * (1 - :vector_weight))) * 100) AS vector_contribution_percent,
            PRINTF("%.1f%%", (COALESCE(1.0 / (:rrf_k + tr.rank), 0) * (1 - :vector_weight)) / ((COALESCE(1.0 / (:rrf_k + vr.rank), 0) * :vector_weight) + (COALESCE(1.0 / (:rrf_k + tr.rank), 0) * (1 - :vector_weight))) * 100) AS text_contribution_percent,
            vr.rank AS vector_rank,
            COALESCE(tr.rank, "-") AS text_rank
        FROM articles a
        LEFT JOIN vector_ranks vr ON a.id = vr.id
        LEFT JOIN text_ranks tr ON a.id = tr.id
        ORDER BY score DESC
        LIMIT :max_results;
    """, {
        'query': QUERY,
        'embedding': sqlite_vec.serialize_float32(query_embedding),
        'max_results': 20,
        'vector_title_weight': 0.7,
        'text_title_multiplier': 3.0,
        'vector_weight': 0.3,
        'rrf_k': 60
    })

    hybrid_search_df = pd.DataFrame(cursor, columns=["Id", "Tytuł", "Sekcja", "Link", "Wynik RRF (0-1)", "Wektory - udział", "Tekst - udział", "Wektory - ranga", "Tekst - ranga"])

hybrid_search_df

,Id,Tytuł,Sekcja,Link,Wynik RRF (0-1),Wektory - udział,Tekst - udział,Wektory - ranga,Tekst - ranga
0,44,England 0-0 Portugal: Lionesses draw with Portugal in World Cup send-off - BBC Sport,None,http://www.bbc.co.uk/sport/football/66065939,0.016393,30.0%,70.0%,1,1
1,510,England U21 1-0 Spain U21: England win Under-21 Euros for first time in 39 years - BBC Sport,None,http://www.bbc.co.uk/sport/football/66127349,0.014839,32.6%,67.4%,2,10
2,439,Edwin van der Sar: Former Manchester United and Ajax goalkeeper in intensive care - BBC Sport,None,http://www.bbc.co.uk/sport/football/66138530,0.013879,32.8%,67.2%,6,15
3,200,Hannah Dingley: Forest Green Rovers name first female boss of a men's professional football side - BBC Sport,None,http://www.bbc.co.uk/sport/football/66105649,0.013736,34.7%,65.3%,3,18
4,33,Equity in Cricket: Prime Minister Rishi Sunak says ICEC report should be 'reset moment' for sport - BBC Sport,None,http://www.bbc.co.uk/sport/cricket/66076354,0.013626,21.0%,79.0%,45,5
5,54,NHS England head urges football clubs to consider gambling ad impact - BBC News,Health,http://www.bbc.co.uk/news/health-66076015,0.013558,32.1%,67.9%,9,16
6,353,Gary Neville admits error in £400m Manchester development - BBC News,Manchester,http://www.bbc.co.uk/news/uk-england-manchester-66114735,0.013278,21.3%,78.7%,46,7
7,43,"Laura Kuenssberg: Love it or hate it, the NHS is here to stay - BBC News",UK Politics,http://www.bbc.co.uk/news/uk-politics-66068224,0.013254,16.2%,83.8%,80,3
8,250,Leicester City facing £880k fine for fixing replica kit prices - BBC News,Leicester,http://www.bbc.co.uk/news/uk-england-leicestershire-66110620,0.013118,26.9%,73.1%,25,13
9,18,"France shooting: Who was Nahel M, shot by French police in Nanterre? - BBC News",Europe,http://www.bbc.co.uk/news/world-europe-66052104,0.012666,10.9%,89.1%,158,2
